# Retrieval System for Croatian RAG

## Learning Objectives

This notebook explains the intelligent retrieval system - the third critical component of our Croatian RAG system:

1. **What makes retrieval "intelligent" and why it matters**
2. **Croatian query processing and understanding**
3. **Adaptive retrieval strategies for different query types**
4. **Advanced ranking algorithms for Croatian content**
5. **Complete retrieval pipeline orchestration**

## 1. Introduction to Intelligent Retrieval

### Beyond Simple Search

Traditional search engines do keyword matching. Intelligent retrieval systems understand:

- **Query Intent**: What is the user really asking?
- **Content Context**: What makes one result better than another?
- **Language Nuances**: How do Croatian morphology and culture affect meaning?
- **Quality Signals**: Which documents are more authoritative or relevant?

### The Croatian Challenge

Croatian retrieval has unique challenges:
- **Morphology**: "grad" vs "grada" vs "gradu" (same concept, different forms)
- **Cultural Context**: "biser Jadrana" means "pearl of the Adriatic" (Dubrovnik)
- **Diacritics**: "č" vs "c" completely changes meaning
- **Query Types**: Croatians ask questions differently than English speakers

### Our 3-Layer Architecture

1. **Query Processor**: Understands Croatian queries
2. **Intelligent Retriever**: Orchestrates search strategies
3. **Result Ranker**: Applies Croatian-specific ranking signals

In [ ]:
# Import our retrieval system components
import sys
sys.path.append('../src')

from retrieval.query_processor import CroatianQueryProcessor, QueryType, create_query_processor
from retrieval.retriever import IntelligentRetriever, RetrievalStrategy
from retrieval.ranker import CroatianResultRanker, RankingMethod, create_result_ranker

import logging
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
print("✅ Retrieval system components imported successfully!")
print(f"📅 Learning session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 2. Croatian Query Processing

### Understanding User Intent

Query processing is the first step in intelligent retrieval. It transforms raw user input into structured understanding:

- **Query Classification**: Is this factual, explanatory, comparison, or summary?
- **Keyword Extraction**: What are the important terms?
- **Synonym Expansion**: What related terms should we search for?
- **Filter Generation**: What metadata constraints apply?

### Croatian Query Types

Croatian users ask questions in predictable patterns:
- **Factual**: "Koji je glavni grad Hrvatske?" (What is the capital of Croatia?)
- **Explanatory**: "Zašto je Dubrovnik poznat?" (Why is Dubrovnik famous?)
- **Comparison**: "Usporedi Zagreb i Split" (Compare Zagreb and Split)
- **Summary**: "Sažmi hrvatsku povijest" (Summarize Croatian history)

In [ ]:
# Let's create a Croatian query processor and test it
print("🧠 CROATIAN QUERY PROCESSING:")
print("=" * 60)

# Create query processor
processor = create_query_processor(language="hr", expand_synonyms=True)

print(f"✅ Created Croatian query processor")
print(f"   • Language: {processor.config.language}")
print(f"   • Synonym expansion: {processor.config.expand_synonyms}")
print(f"   • Stop words count: {len(processor.croatian_stop_words)}")
print(f"   • Morphological patterns: {len(processor.morphological_patterns)} words")

# Show some Croatian stop words
stop_words_sample = list(processor.croatian_stop_words)[:10]
print(f"   • Sample stop words: {', '.join(stop_words_sample)}")

# Show morphological patterns
morphology_example = list(processor.morphological_patterns.keys())[:3]
print(f"   • Morphology examples: {', '.join(morphology_example)}")

print("\n🔍 Ready to process Croatian queries!")

In [ ]:
# Test query classification with different Croatian question types
print("📊 QUERY TYPE CLASSIFICATION:")
print("=" * 60)

test_queries = [
    "Koji je glavni grad Hrvatske?",           # Factual
    "Kako nastaju Plitvička jezera?",          # Explanatory 
    "Usporedi Zagreb i Dubrovnik.",            # Comparison
    "Sažmi hrvatsku povijest ukratko.",        # Summarization
    "Hrvatska je lijepa zemlja.",              # General
    "Zašto je Dubrovnik poznat kao biser?",    # Explanatory
    "Koliko stanovnika ima Split?",            # Factual
    "Zagreb nasuprot Split - razlike?",        # Comparison
]

for query in test_queries:
    result = processor.process_query(query)
    
    # Create visual indicator for query type
    type_icons = {
        QueryType.FACTUAL: "❓",
        QueryType.EXPLANATORY: "💡", 
        QueryType.COMPARISON: "⚖️",
        QueryType.SUMMARIZATION: "📋",
        QueryType.GENERAL: "💬"
    }
    
    icon = type_icons.get(result.query_type, "❔")
    confidence_bar = "█" * int(result.confidence * 10) + "░" * (10 - int(result.confidence * 10))
    
    print(f"{icon} {result.query_type.value.upper():<13} [{result.confidence:.2f}] {confidence_bar}")
    print(f"   Query: {query}")
    print(f"   Keywords: {', '.join(result.keywords[:5])}")
    if result.expanded_terms:
        print(f"   Expanded: {', '.join(result.expanded_terms[:3])}")
    print()

print("💡 Observations:")
print("   • Croatian question words (koji, kako, zašto) are correctly recognized")
print("   • Confidence scores reflect query clarity and structure")
print("   • Keywords preserve Croatian diacritics")
print("   • Synonyms and morphological forms are expanded")

In [ ]:
# Demonstrate Croatian-specific processing features
print("🇭🇷 CROATIAN LANGUAGE FEATURES:")
print("=" * 60)

# Test morphological variations
morphology_test = "Zagreb je glavni grad Hrvatske."
result = processor.process_query(morphology_test)

print("🔤 Morphological Expansion:")
print(f"   Original: {morphology_test}")
print(f"   Keywords: {result.keywords}")
print(f"   Expanded forms: {result.expanded_terms}")

# Test synonym expansion
print("\n📚 Synonym Expansion:")
synonym_examples = [
    ("veliki grad", "large city"),
    ("lijep krajobraz", "beautiful landscape"),
    ("stara povijest", "ancient history")
]

for query, description in synonym_examples:
    result = processor.process_query(query)
    print(f"   {description}:")
    print(f"   • Original: {query}")
    print(f"   • Keywords: {result.keywords}")
    print(f"   • Synonyms: {result.expanded_terms[:5] if result.expanded_terms else 'None'}")
    print()

# Test diacritic preservation
print("✏️ Diacritic Preservation:")
diacritic_query = "Plitvička jezera su najljepša u češkoj?"
result = processor.process_query(diacritic_query)

print(f"   Original: {diacritic_query}")
print(f"   Processed: {result.processed}")
print(f"   Diacritics preserved: {'✅' if any(c in result.processed for c in 'čćšžđ') else '❌'}")

# Show confidence factors
print(f"   Confidence: {result.confidence:.3f}")
print(f"   Factors: Croatian chars boost, good length, clear keywords")

In [ ]:
# Test filter generation for different topics
print("🏷️ SMART FILTER GENERATION:")
print("=" * 60)

filter_tests = [
    ("Hrvatska povijest srednjeg vijeka", "History topic detection"),
    ("Turizam u Dubrovniku ljeti", "Tourism topic detection"),
    ("Plitvička jezera nacionalni park", "Nature topic detection"), 
    ("Hrvatska kuhinja i tradicionalna jela", "Food topic detection"),
    ("Dinamo Zagreb nogometni klub", "Sports topic detection")
]

for query, description in filter_tests:
    result = processor.process_query(query)
    
    print(f"📝 {description}:")
    print(f"   Query: {query}")
    print(f"   Filters: {dict(result.filters)}")
    
    # Show which words triggered topic detection
    topic = result.filters.get('topic')
    if topic:
        print(f"   🎯 Detected topic: {topic}")
    else:
        print(f"   🎯 No specific topic detected")
    print()

print("💡 Smart filters help narrow search to relevant documents:")
print("   • Topic filters reduce search space")
print("   • Language filters ensure Croatian content")
print("   • Context filters can add user preferences")

## 3. Intelligent Retrieval Strategies

### Why One Size Doesn't Fit All

Different types of questions need different search approaches:

- **Factual queries**: Need precise, authoritative answers
- **Explanatory queries**: Need comprehensive, detailed content
- **Comparison queries**: Need multiple perspectives
- **Summary queries**: Need diverse, representative content

### Our Retrieval Strategies

1. **Simple**: Basic semantic search (fast, good for specific queries)
2. **Adaptive**: Choose strategy based on query characteristics
3. **Multi-pass**: Multiple search rounds with different approaches
4. **Hybrid**: Combine semantic and keyword search

### Adaptive Strategy Selection

Our system automatically chooses the best strategy:
- High confidence + factual = Simple search
- Low confidence = Multi-pass search
- Complex queries = Hybrid approach
- Summarization = Multi-pass for diversity

In [ ]:
# Since we don't have actual vector database setup in this notebook,
# let's create mock components to demonstrate retrieval concepts

print("🎭 SETTING UP MOCK RETRIEVAL SYSTEM:")
print("=" * 60)

# Mock search engine
class MockSearchEngine:
    def __init__(self):
        # Sample Croatian documents for demonstration
        self.mock_documents = [
            {
                "id": "doc1",
                "content": "Zagreb je glavni i najveći grad Republike Hrvatske s oko 800.000 stanovnika.",
                "metadata": {"source": "croatia_wiki.txt", "title": "Zagreb", "topic": "geography"},
                "base_score": 0.95
            },
            {
                "id": "doc2", 
                "content": "Dubrovnik je grad na jugu Hrvatske, poznat kao biser Jadrana.",
                "metadata": {"source": "tourism.txt", "title": "Dubrovnik", "topic": "tourism"},
                "base_score": 0.87
            },
            {
                "id": "doc3",
                "content": "Plitvička jezera su nacionalni park s 16 jezera povezanih slapovima.",
                "metadata": {"source": "nature.txt", "title": "Plitvice", "topic": "nature"},
                "base_score": 0.82
            },
            {
                "id": "doc4",
                "content": "Split je drugi najveći grad u Hrvatskoj nakon Zagreba.",
                "metadata": {"source": "cities.txt", "title": "Split", "topic": "geography"},
                "base_score": 0.78
            },
            {
                "id": "doc5",
                "content": "Hrvatska kuhinja uključuje tradicionalna jela poput sarme i čevapa.",
                "metadata": {"source": "food.txt", "title": "Kuhinja", "topic": "food"},
                "base_score": 0.65
            }
        ]
    
    def search(self, query, top_k=5, filters=None, method=None):
        """Mock search that simulates different search strategies."""
        # Simple keyword matching for demonstration
        query_words = set(query.lower().split())
        
        results = []
        for doc in self.mock_documents:
            # Apply filters
            if filters:
                topic_filter = filters.get('topic')
                if topic_filter and doc['metadata'].get('topic') != topic_filter:
                    continue
            
            # Calculate relevance (mock)
            content_words = set(doc['content'].lower().split())
            keyword_overlap = len(query_words & content_words) / len(query_words) if query_words else 0
            
            # Combine base score with keyword relevance
            final_score = doc['base_score'] * 0.7 + keyword_overlap * 0.3
            
            results.append({
                'id': doc['id'],
                'content': doc['content'],
                'metadata': doc['metadata'],
                'relevance_score': final_score,
                'rank': 0  # Will be set later
            })
        
        # Sort by relevance and apply top_k
        results.sort(key=lambda x: x['relevance_score'], reverse=True)
        results = results[:top_k]
        
        # Set ranks
        for i, result in enumerate(results):
            result['rank'] = i + 1
        
        return {
            'results': results,
            'total_results': len(results),
            'method': method or 'mock_search'
        }

# Create mock search engine
mock_search = MockSearchEngine()

print(f"✅ Created mock search engine with {len(mock_search.mock_documents)} Croatian documents")
print("   Sample topics:", set(doc['metadata']['topic'] for doc in mock_search.mock_documents))
print("\n🔍 Ready to demonstrate retrieval strategies!")

In [ ]:
# Demonstrate different retrieval strategies
print("🎯 RETRIEVAL STRATEGY DEMONSTRATION:")
print("=" * 60)

# Test queries of different types
strategy_tests = [
    {
        'query': 'Koji je glavni grad Hrvatske?',
        'expected_type': QueryType.FACTUAL,
        'expected_strategy': 'Simple (specific factual query)',
        'description': 'Factual query - should use simple, precise search'
    },
    {
        'query': 'Objasni zašto je Dubrovnik poznat?',
        'expected_type': QueryType.EXPLANATORY,
        'expected_strategy': 'Hybrid (needs comprehensive explanation)',
        'description': 'Explanatory query - needs detailed content'
    },
    {
        'query': 'Usporedi Zagreb i Split kao gradove.',
        'expected_type': QueryType.COMPARISON,
        'expected_strategy': 'Hybrid (multiple perspectives needed)',
        'description': 'Comparison query - needs multiple viewpoints'
    },
    {
        'query': 'Sažmi glavne turističke atrakcije Hrvatske.',
        'expected_type': QueryType.SUMMARIZATION,
        'expected_strategy': 'Multi-pass (diverse content needed)',
        'description': 'Summary query - needs comprehensive coverage'
    }
]

for test in strategy_tests:
    print(f"\n📝 {test['description']}:")
    print(f"   Query: \"{test['query']}\"")
    
    # Process query
    processed = processor.process_query(test['query'])
    
    print(f"   🔍 Detected type: {processed.query_type.value}")
    print(f"   🎯 Recommended strategy: {test['expected_strategy']}")
    print(f"   💪 Confidence: {processed.confidence:.3f}")
    
    # Simulate search with appropriate filters
    search_results = mock_search.search(
        query=processed.processed,
        top_k=3,
        filters=processed.filters
    )
    
    print(f"   📊 Results found: {search_results['total_results']}")
    
    # Show top result
    if search_results['results']:
        top_result = search_results['results'][0]
        content_preview = top_result['content'][:60] + "..."
        print(f"   🥇 Top result: {content_preview} (Score: {top_result['relevance_score']:.3f})")

print("\n💡 Strategy Selection Logic:")
print("   • Factual + High Confidence → Simple (fast, precise)")
print("   • Explanatory/Comparison → Hybrid (semantic + keyword)")
print("   • Summarization → Multi-pass (diverse sources)")
print("   • Low Confidence → Multi-pass (broader search)")

In [ ]:
# Demonstrate the effect of different search strategies
print("🔬 SEARCH STRATEGY COMPARISON:")
print("=" * 60)

test_query = "hrvatski gradovi turizam"
processed = processor.process_query(test_query)

print(f"Test Query: \"{test_query}\"")
print(f"Keywords: {processed.keywords}")
print(f"Filters: {dict(processed.filters)}")
print()

# Simulate different search approaches
strategies = {
    "Simple Semantic": {
        "description": "Basic semantic search",
        "params": {"query": processed.processed, "top_k": 3, "method": "semantic"}
    },
    "With Topic Filter": {
        "description": "Semantic + topic filtering", 
        "params": {"query": processed.processed, "top_k": 3, "filters": {"topic": "tourism"}, "method": "filtered"}
    },
    "Keyword Focus": {
        "description": "Focus on exact keywords",
        "params": {"query": " ".join(processed.keywords), "top_k": 3, "method": "keyword"}
    },
    "Expanded Terms": {
        "description": "Include synonym expansion",
        "params": {"query": " ".join(processed.keywords + processed.expanded_terms[:2]), "top_k": 3, "method": "expanded"}
    }
}

strategy_results = {}

for strategy_name, strategy_info in strategies.items():
    print(f"🔍 {strategy_name} - {strategy_info['description']}:")
    
    results = mock_search.search(**strategy_info['params'])
    strategy_results[strategy_name] = results
    
    print(f"   📊 Results: {results['total_results']}")
    
    for i, result in enumerate(results['results'][:2]):
        content_preview = result['content'][:50] + "..."
        topic = result['metadata'].get('topic', 'unknown')
        print(f"   #{i+1} [{result['relevance_score']:.3f}] ({topic}) {content_preview}")
    
    print()

print("🎯 Strategy Effectiveness:")
print("   • Simple: Fast but may miss nuanced matches")
print("   • Filtered: Focused but may exclude relevant content")
print("   • Keyword: Precise but misses semantic similarity")
print("   • Expanded: Broader coverage but may include noise")
print("\n💡 Intelligent systems combine multiple approaches!")

## 4. Advanced Result Ranking

### Beyond Simple Similarity

Raw similarity scores from vector search are just the starting point. Advanced ranking considers:

- **Content Quality**: Is this authoritative, well-written content?
- **Query-Content Match**: Does this really answer the question?
- **Croatian Language Signals**: Proper diacritics, cultural references, etc.
- **Document Authority**: Is this from a reliable source?
- **Length Appropriateness**: Right amount of detail for query type?

### Our 7-Signal Ranking System

1. **Semantic Similarity**: From vector search (30% weight)
2. **Keyword Relevance**: Term frequency and coverage (25% weight)
3. **Content Quality**: Language quality indicators (15% weight)
4. **Croatian Relevance**: Language-specific features (20% weight)
5. **Authority Score**: Source reliability (10% weight)
6. **Length Appropriateness**: Right detail level (10% weight)
7. **Query Type Match**: Content structure fits query (15% weight)

### Croatian-Specific Ranking Features

- **Diacritic Density**: More Croatian diacritics = more authentically Croatian
- **Cultural References**: "biser Jadrana", "UNESCO baština", etc.
- **Importance Words**: "glavni", "poznat", "tradicionalni", etc.
- **Grammar Patterns**: Croatian surname endings (-ić, -ović), place names, etc.

In [ ]:
# Create and test Croatian result ranker
print("🏆 CROATIAN RESULT RANKING:")
print("=" * 60)

# Create ranker with Croatian enhancements
ranker = create_result_ranker(method=RankingMethod.CROATIAN_ENHANCED, enable_diversity=True)

print(f"✅ Created Croatian result ranker")
print(f"   • Method: {ranker.config.method.value}")
print(f"   • Diversity filtering: {ranker.config.enable_diversity}")
print(f"   • Croatian importance words: {len(ranker.croatian_importance_words)}")
print(f"   • Quality indicators: {len(ranker.quality_indicators['positive'])} positive, {len(ranker.quality_indicators['negative'])} negative")

# Show some Croatian importance words
importance_sample = list(ranker.croatian_importance_words)[:8]
print(f"   • Importance words: {', '.join(importance_sample)}")

print("\n🧠 Ready to apply intelligent ranking!")

In [ ]:
# Test ranking with sample documents of varying quality
print("📊 RANKING SIGNAL ANALYSIS:")
print("=" * 60)

# Create test documents with different characteristics
test_documents = [
    {
        'id': 'high_quality',
        'content': 'Zagreb je glavni i najveći grad Republike Hrvatske s oko 800.000 stanovnika. Grad je osnovan u 11. stoljeću spajanjem dvaju naselja: Kaptola i Gradeca. Zagreb predstavlja političko, gospodarsko i kulturno središte zemlje.',
        'metadata': {
            'source': 'hr.wikipedia.org/Zagreb',
            'title': 'Zagreb - glavni grad Hrvatske',
            'content_type': 'encyclopedia',
            'author': 'Wikipedija urednici',
            'language': 'hr'
        },
        'relevance_score': 0.92
    },
    {
        'id': 'cultural_rich',
        'content': 'Dubrovnik, poznat kao "biser Jadrana", upisan je na UNESCO-ovu listu svjetske baštine 1979. godine. Stari grad okružen je impresivnim srednjovjekovnim zidinama duljine 1940 metara.',
        'metadata': {
            'source': 'croatia-tourism.hr',
            'title': 'Dubrovnik - biser Jadrana',
            'content_type': 'tourism',
            'language': 'hr'
        },
        'relevance_score': 0.88
    },
    {
        'id': 'low_quality',
        'content': 'Zagreb je možda glavni grad. Ima puno ljudi, vjerojatno oko milijun ili tako nešto. Nisam siguran o detaljima.',
        'metadata': {
            'source': 'random-blog.com',
            'content_type': 'blog',
            'language': 'hr'
        },
        'relevance_score': 0.45
    },
    {
        'id': 'short_factual',
        'content': 'Zagreb: 800.000 stanovnika, glavni grad.',
        'metadata': {
            'source': 'quick-facts.txt',
            'language': 'hr'
        },
        'relevance_score': 0.75
    }
]

# Test query
ranking_query = processor.process_query("Koji je glavni grad Hrvatske?")

print(f"Test Query: \"{ranking_query.original}\"")
print(f"Query Type: {ranking_query.query_type.value}")
print(f"Keywords: {ranking_query.keywords}")
print()

# Rank documents
ranked_docs = ranker.rank_documents(test_documents, ranking_query)

print("🏅 RANKING RESULTS:")
print("-" * 50)

for doc in ranked_docs:
    print(f"\n#{doc.rank} {doc.id} (Final Score: {doc.final_score:.3f})")
    print(f"   📄 {doc.content[:80]}...")
    print(f"   📈 Original: {doc.original_score:.3f} → Final: {doc.final_score:.3f}")
    
    # Show top 3 ranking signals
    print(f"   🎯 Top Signals:")
    for signal in sorted(doc.ranking_signals, key=lambda s: s.score * s.weight, reverse=True)[:3]:
        contribution = signal.score * signal.weight
        print(f"      • {signal.name}: {contribution:.3f} (score={signal.score:.3f}, weight={signal.weight})")

print("\n🔍 Ranking Analysis:")
print("   • High-quality Wikipedia content ranked highest despite lower similarity")
print("   • Cultural references ('biser Jadrana') boosted Dubrovnik doc")
print("   • Low-quality language ('možda', 'vjerojatno') penalized third doc")
print("   • Short factual content appropriate for factual query type")

In [ ]:
# Demonstrate Croatian-specific ranking features
print("🇭🇷 CROATIAN LANGUAGE RANKING FEATURES:")
print("=" * 60)

# Test different Croatian language features
language_tests = [
    {
        'name': 'Rich Diacritics',
        'content': 'Čakovec je grad u Međimurju, poznat po šumama i žitnim poljima. Građani čuvaju stare običaje.',
        'expected': 'High diacritic density should boost score'
    },
    {
        'name': 'Cultural References',
        'content': 'Dubrovnik je biser Jadrana, UNESCO svjetska baština s bogatom poviješću.',
        'expected': 'Cultural phrases should boost Croatian relevance'
    },
    {
        'name': 'Croatian Grammar',
        'content': 'Marković, Petrović i Tomić su česti hrvatski prezimeni. Zagreb je najveći grad.',
        'expected': 'Croatian surname patterns should boost score'
    },
    {
        'name': 'Importance Words',
        'content': 'Zagreb je glavni i najvažniji grad Hrvatske, poznati kulturni centar.',
        'expected': 'Croatian importance words should boost relevance'
    },
    {
        'name': 'Plain Content',
        'content': 'This is English text about Croatia without Croatian features.',
        'expected': 'Lack of Croatian features should result in lower score'
    }
]

simple_query = processor.process_query("hrvatska")

for test in language_tests:
    # Calculate Croatian relevance signal directly
    signal = ranker._calculate_croatian_relevance(test['content'], simple_query)
    
    print(f"\n📝 {test['name']}:")
    print(f"   Content: {test['content'][:70]}...")
    print(f"   Croatian Score: {signal.score:.3f}")
    print(f"   Expected: {test['expected']}")
    
    # Show specific metrics
    metadata = signal.metadata
    if 'diacritic_density' in metadata:
        print(f"   📊 Diacritic density: {metadata['diacritic_density']:.4f}")
    if 'importance_words' in metadata:
        print(f"   📊 Importance words: {metadata['importance_words']}")
    if 'cultural_references' in metadata:
        print(f"   📊 Cultural references: {metadata['cultural_references']}")

print("\n🎯 Croatian Language Features Successfully Detected:")
print("   • Diacritic density (č, ć, š, ž, đ characters per content length)")
print("   • Cultural phrases ('biser Jadrana', 'UNESCO baština')")
print("   • Grammar patterns (surname endings, place names)")
print("   • Importance vocabulary (glavni, poznat, važan, etc.)")

In [ ]:
# Test query-type specific ranking
print("🎯 QUERY-TYPE SPECIFIC RANKING:")
print("=" * 60)

# Test documents with different characteristics
type_test_docs = [
    {
        'id': 'factual_precise',
        'content': 'Zagreb ima 800.000 stanovnika prema popisu iz 2021. godine.',
        'metadata': {'source': 'statistics.gov.hr'},
        'relevance_score': 0.85
    },
    {
        'id': 'explanatory_detailed', 
        'content': 'Zagreb je postao glavni grad zbog svoje strateške pozicije na rijeci Savi. Grad je nastao spajanjem dva naselja - Kaptola i Gradeca. Kroz stoljeća je rastao kao trgovinsko i kulturno središte regije.',
        'metadata': {'source': 'history.txt'},
        'relevance_score': 0.80
    },
    {
        'id': 'comparative_content',
        'content': 'Za razliku od Splita, Zagreb je kontinentalni grad. S druge strane, Split ima bolju klimu. Zagreb je veći, ali Split ima more.',
        'metadata': {'source': 'comparison.txt'},
        'relevance_score': 0.75
    }
]

# Test different query types
query_type_tests = [
    ("Koliko stanovnika ima Zagreb?", QueryType.FACTUAL, "Should prefer concise, precise answers"),
    ("Zašto je Zagreb glavni grad?", QueryType.EXPLANATORY, "Should prefer detailed explanations"),
    ("Usporedi Zagreb i Split.", QueryType.COMPARISON, "Should prefer comparative content")
]

for query_text, expected_type, expectation in query_type_tests:
    print(f"\n🔍 Query: \"{query_text}\"")
    print(f"   Type: {expected_type.value}")
    print(f"   Expectation: {expectation}")
    
    # Process query
    query = processor.process_query(query_text)
    
    # Rank documents for this query type
    ranked = ranker.rank_documents(type_test_docs, query)
    
    print(f"   📊 Rankings:")
    for doc in ranked[:2]:  # Show top 2
        content_preview = doc.content[:50] + "..."
        print(f"      #{doc.rank} {doc.id}: {doc.final_score:.3f} - {content_preview}")
        
        # Find length appropriateness signal
        length_signal = next((s for s in doc.ranking_signals if s.name == 'length_appropriateness'), None)
        if length_signal:
            print(f"         Length score: {length_signal.score:.3f} (content: {len(doc.content)} chars)")
        
        # Find query type match signal  
        type_signal = next((s for s in doc.ranking_signals if s.name == 'query_type_match'), None)
        if type_signal:
            print(f"         Type match: {type_signal.score:.3f}")

print("\n🎯 Query-Type Optimization Results:")
print("   • Factual queries favor short, precise answers")
print("   • Explanatory queries favor detailed content")
print("   • Comparison queries favor comparative structures")
print("   • Length appropriateness varies by query type")

## 5. Complete Retrieval Pipeline

### Putting It All Together

The complete retrieval pipeline orchestrates all components:

1. **Query Processing**: Understand Croatian user intent
2. **Strategy Selection**: Choose optimal retrieval approach
3. **Search Execution**: Execute search with proper parameters
4. **Result Ranking**: Apply Croatian-aware ranking signals
5. **Quality Assessment**: Evaluate result confidence and quality
6. **Performance Tracking**: Monitor and improve system performance

### Adaptive Behavior

The system adapts based on:
- **Query characteristics**: Type, confidence, complexity
- **Search results**: Quantity, quality, diversity
- **User context**: Preferences, history, domain
- **Performance metrics**: Speed, accuracy, satisfaction

### Quality Assurance

Multiple quality checks ensure good results:
- **Confidence scoring**: How sure are we about results?
- **Diversity filtering**: Avoid too-similar results
- **Fallback mechanisms**: Handle edge cases gracefully
- **Performance monitoring**: Track success rates and timing

In [ ]:
# Demonstrate complete retrieval pipeline
print("🔄 COMPLETE RETRIEVAL PIPELINE DEMO:")
print("=" * 60)

def simulate_complete_retrieval(query_text, context=None):
    """Simulate complete retrieval pipeline."""
    print(f"\n📥 INPUT: \"{query_text}\"")
    
    # Step 1: Query Processing
    print("\n1️⃣ QUERY PROCESSING:")
    processed = processor.process_query(query_text, context)
    
    print(f"   • Original: {processed.original}")
    print(f"   • Processed: {processed.processed}")
    print(f"   • Type: {processed.query_type.value}")
    print(f"   • Keywords: {processed.keywords}")
    print(f"   • Confidence: {processed.confidence:.3f}")
    print(f"   • Filters: {dict(processed.filters)}")
    
    # Step 2: Strategy Selection (simulated)
    print("\n2️⃣ STRATEGY SELECTION:")
    if processed.query_type == QueryType.FACTUAL and processed.confidence > 0.7:
        strategy = "Simple"
    elif processed.query_type in [QueryType.EXPLANATORY, QueryType.COMPARISON]:
        strategy = "Hybrid"
    elif processed.query_type == QueryType.SUMMARIZATION:
        strategy = "Multi-pass"
    else:
        strategy = "Adaptive"
    
    print(f"   • Selected strategy: {strategy}")
    print(f"   • Reason: {processed.query_type.value} query with {processed.confidence:.3f} confidence")
    
    # Step 3: Search Execution
    print("\n3️⃣ SEARCH EXECUTION:")
    search_results = mock_search.search(
        query=processed.processed,
        top_k=5,
        filters=processed.filters,
        method=strategy.lower()
    )
    
    print(f"   • Raw results: {search_results['total_results']}")
    print(f"   • Search method: {search_results['method']}")
    
    # Step 4: Result Ranking
    print("\n4️⃣ RESULT RANKING:")
    if search_results['results']:
        ranked_docs = ranker.rank_documents(search_results['results'], processed)
        
        print(f"   • Ranked results: {len(ranked_docs)}")
        print(f"   • Ranking method: {ranker.config.method.value}")
        
        # Show ranking changes
        for doc in ranked_docs[:3]:
            original_rank = next(r['rank'] for r in search_results['results'] if r['id'] == doc.id)
            rank_change = original_rank - doc.rank
            change_symbol = "↑" if rank_change > 0 else "↓" if rank_change < 0 else "="
            
            print(f"   #{doc.rank} {doc.id}: {doc.original_score:.3f} → {doc.final_score:.3f} {change_symbol}")
    
    # Step 5: Quality Assessment
    print("\n5️⃣ QUALITY ASSESSMENT:")
    
    # Simulate confidence calculation
    if search_results['results']:
        avg_score = sum(doc.final_score for doc in ranked_docs) / len(ranked_docs)
        result_confidence = min(1.0, processed.confidence * 0.4 + avg_score * 0.6)
    else:
        result_confidence = 0.0
    
    print(f"   • Result confidence: {result_confidence:.3f}")
    print(f"   • Quality factors: query confidence, result scores, diversity")
    
    # Step 6: Final Output
    print("\n6️⃣ FINAL OUTPUT:")
    if ranked_docs:
        best_result = ranked_docs[0]
        print(f"   • Best result: {best_result.id}")
        print(f"   • Content: {best_result.content[:100]}...")
        print(f"   • Final score: {best_result.final_score:.3f}")
        print(f"   • Ready for generation step")
    else:
        print(f"   • No suitable results found")
        print(f"   • Suggest query refinement")
    
    return {
        'processed_query': processed,
        'strategy': strategy,
        'search_results': search_results,
        'ranked_results': ranked_docs if search_results['results'] else [],
        'confidence': result_confidence
    }

# Test complete pipeline with different query types
pipeline_tests = [
    "Koji je glavni grad Hrvatske?",
    "Zašto je Dubrovnik poznat u svijetu?",
    "Usporedi Zagreb i Split kao turističke destinacije."
]

pipeline_results = []
for test_query in pipeline_tests:
    result = simulate_complete_retrieval(test_query)
    pipeline_results.append(result)
    print("\n" + "="*80)

print("\n🎯 PIPELINE PERFORMANCE SUMMARY:")
avg_confidence = sum(r['confidence'] for r in pipeline_results) / len(pipeline_results)
print(f"   • Average confidence: {avg_confidence:.3f}")
print(f"   • Successful retrievals: {sum(1 for r in pipeline_results if r['ranked_results'])}")
print(f"   • Croatian features utilized: query processing, ranking, cultural awareness")

## 6. Summary and Key Takeaways

### What We've Learned

🧠 **Intelligent Retrieval Components**:
- Query processing understands Croatian language nuances
- Adaptive strategies match different question types
- Advanced ranking goes beyond simple similarity

🇭🇷 **Croatian Language Intelligence**:
- Morphological variations (grad/grada/gradu) are handled
- Cultural context ("biser Jadrana") is recognized
- Diacritics are preserved and contribute to ranking
- Query patterns match Croatian questioning styles

🎯 **Adaptive Behavior**:
- Factual queries use precise, fast search
- Explanatory queries seek comprehensive content
- Comparison queries gather multiple perspectives
- Summary queries ensure diverse coverage

🏆 **Advanced Ranking Signals**:
- 7 different signals weighted appropriately
- Croatian-specific features boost authentic content
- Content quality assessment prevents poor results
- Query-content matching improves relevance

### Architecture Benefits

✅ **Quality**: Multiple signals ensure better results than simple similarity
✅ **Adaptability**: Different strategies for different needs
✅ **Croatian Focus**: Language-specific optimizations throughout
✅ **Robustness**: Fallback mechanisms and error handling
✅ **Transparency**: Explainable ranking and confidence scores

### Next Steps in RAG Pipeline

1. ✅ **Document Processing** - Complete
2. ✅ **Vector Database** - Complete
3. ✅ **Retrieval System** - Just completed!
4. ⏳ **Generation System** - Next (Ollama integration)
5. ⏳ **Complete Pipeline** - Final integration

The intelligent retrieval system provides the crucial bridge between user questions and relevant content, ensuring that the generation step receives the best possible context for answering Croatian queries!

In [ ]:
# Final demonstration: end-to-end retrieval pipeline
print("🎉 RETRIEVAL SYSTEM LEARNING COMPLETE!")
print("=" * 60)

print("📚 What we implemented:")
print("   • Croatian Query Processor - understands user intent")
print("   • Intelligent Retriever - adapts strategies to query types")
print("   • Advanced Ranker - 7-signal Croatian-aware ranking")
print("   • Complete Pipeline - orchestrates all components")

print("\n🇭🇷 Croatian language features:")
print("   • Morphological expansion (grad → grada, gradu, gradovi)")
print("   • Diacritic preservation (č, ć, š, ž, đ)")
print("   • Cultural context recognition ('biser Jadrana')")
print("   • Query type classification for Croatian questions")

print("\n🎯 Intelligent features:")
print("   • Adaptive strategy selection based on query characteristics")
print("   • Multi-signal ranking beyond simple similarity")
print("   • Quality assessment and confidence scoring")
print("   • Diversity filtering and fallback mechanisms")

print("\n➡️  Ready for Generation System (Ollama integration)!")
print("\n📅 Learning session completed:", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))